In [1]:
spark = SparkSession.builder.appName("pysparkClustering").getOrCreate()

In [2]:
df = spark.read.csv('vgsales.csv',inferSchema=True,header='true')

In [3]:
df.columns

['Rank',
 'Name',
 'Platform',
 'Year',
 'Genre',
 'Publisher',
 'NA_Sales',
 'EU_Sales',
 'JP_Sales',
 'Other_Sales',
 'Global_Sales']

In [4]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['NA_Sales','EU_Sales','JP_Sales'],
                            outputCol='features')
data = assembler.transform(df)

K-Means

In [5]:
from pyspark.ml.clustering import KMeans
kmeans = KMeans().setK(4).setSeed(1)
model = kmeans.fit(data)

In [6]:
predictions = model.transform(data)

In [7]:
predictions.select('features','prediction').toPandas()

,features,prediction
0,"[41.49, 29.02, 3.77]",1
1,"[29.08, 3.58, 6.81]",1
2,"[15.85, 12.88, 3.79]",1
3,"[15.75, 11.01, 3.28]",1
4,"[11.27, 8.89, 10.22]",1
...,...,...
16593,"[0.01, 0.0, 0.0]",0
16594,"[0.01, 0.0, 0.0]",0
16595,"(0.0, 0.0, 0.0)",0
16596,"[0.0, 0.01, 0.0]",0


In [11]:
predictions.select('NA_Sales','EU_Sales','JP_Sales').filter('prediction = 1').summary().toPandas()

,summary,NA_Sales,EU_Sales,JP_Sales
0,count,11,11,11
1,mean,19.867272727272727,8.972727272727273,4.24909090909091
2,stddev,9.372031893788122,7.649181774424099,2.87100489217784
3,min,11.27,0.63,0.24
4,25%,14.03,3.58,2.93
5,50%,15.75,8.89,3.79
6,75%,26.93,11.01,6.5
7,max,41.49,29.02,10.22


In [12]:
from pyspark.ml.evaluation import ClusteringEvaluator
evaluator = ClusteringEvaluator()

In [13]:
silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

Silhouette with squared euclidean distance = 0.8986214827769562


In [14]:
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Cluster Centers: 
[0.13637421 0.07355795 0.05602016]
[19.86727273  8.97272727  4.24909091]
[1.544      0.88545631 0.23396117]
[5.99101266 3.60607595 1.7243038 ]


Latent Dirichlet allocation (LDA)
lambat banget (warning)

In [14]:
from pyspark.ml.clustering import LDA
lda = LDA(k=10, maxIter=10)
model = lda.fit(data)

ll = model.logLikelihood(data)
lp = model.logPerplexity(data)
print("The lower bound on the log likelihood of the entire corpus: " + str(ll))
print("The upper bound on perplexity: " + str(lp))

The lower bound on the log likelihood of the entire corpus: -24613.388494579278
The upper bound on perplexity: 3.03191491784746


In [15]:
topics = model.describeTopics(3)
print("The topics described by their top-weighted terms:")
topics.show(truncate=False)

The topics described by their top-weighted terms:
+-----+-----------+---------------------------------------------------------------+
|topic|termIndices|termWeights                                                    |
+-----+-----------+---------------------------------------------------------------+
|0    |[0, 2, 1]  |[0.38152901494784214, 0.3366416106997798, 0.28182937435237804] |
|1    |[0, 2, 1]  |[0.38248289673851804, 0.33005936947175796, 0.28745773378972395]|
|2    |[1, 0, 2]  |[0.7336031683991809, 0.21337996999861306, 0.05301686160220588] |
|3    |[0, 2, 1]  |[0.4182424631439845, 0.2921478841032863, 0.28960965275272915]  |
|4    |[0, 1, 2]  |[0.7195638139982149, 0.2337436577305689, 0.046692528271216255] |
|5    |[0, 2, 1]  |[0.4237702089542661, 0.3186670331742763, 0.2575627578714576]   |
|6    |[1, 0, 2]  |[0.36616061944851214, 0.33637183729159215, 0.2974675432598958] |
|7    |[1, 0, 2]  |[0.4826063364547157, 0.48206889050717433, 0.03532477303810992] |
|8    |[2, 0, 1]  |[0.8426

In [16]:
transformed = model.transform(data)
transformed.show(truncate=False)

+----+--------------------------------------------+--------+----+------------+----------------------+--------+--------+--------+-----------+------------+------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Rank|Name                                        |Platform|Year|Genre       |Publisher             |NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|features          |topicDistribution                                                                                                                                                                                                     |
+----+--------------------------------------------+--------+----+------------+----------------------+--------+--------+--------+-----------+------------+------------------+----------------------------------------------------

Gaussian Mixture Model (GMM)

In [12]:
from pyspark.ml.clustering import GaussianMixture
gmm = GaussianMixture().setK(2).setSeed(538009335)
model = gmm.fit(data)

In [13]:
print("Gaussians shown as a DataFrame: ")
model.gaussiansDF.show(truncate=False)

Gaussians shown as a DataFrame: 
+-------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|mean                                                         |cov                                                                                                                                                                                                       |
+-------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[0.16734408828256747,0.08257419072082968,0.03442008954599476]|0.20390518066846927   0.08425647918484716   0.023909633075897767  \n0.08425647918484716   0.06230993349